# Практические занятия №12-13 (часть 2)

### Фаззинг (Тестирование на случайных структурированных входных данных)

**4.1.** (1 балл)

Продолжение раздела "Контрактное программирование" из части 1. Напишите тест, доказывающий, что переданная на вход функция коммутативна. Используйте модуль `hypothesis`. Запустите тесты для реализованных ранее функций `euclidean_distance` и `manhattan_distance`.  

In [ ]:
import numpy as np


def euclidean_distance(p1, p2):
    '''
    >>> euclidean_distance([0, 0], [3, 4])
    5.0
    '''
    return float(np.sqrt(sum((p - q) ** 2 for p, q in zip(p1, p2))))


def manhattan_distance(p1, p2):
    '''
    >>> manhattan_distance([0, 0], [3, 4])
    7.0
    '''
    return float(sum(abs(p-q) for p, q in zip(p1, p2)))







In [2]:
import hypothesis.strategies as st
from hypothesis import given
import numpy as np
import ipytest


def euclidean_distance(p1, p2):
    '''
    >>> euclidean_distance([0, 0], [3, 4])
    5.0
    '''
    return float(np.sqrt(sum((p - q) ** 2 for p, q in zip(p1, p2))))


def manhattan_distance(p1, p2):
    '''
    >>> manhattan_distance([0, 0], [3, 4])
    7.0
    '''
    return float(sum(abs(p-q) for p, q in zip(p1, p2)))


@given(
    st.lists(st.floats(min_value=-100, max_value=100), min_size=3, max_size=3),
    st.lists(st.floats(min_value=-100, max_value=100), min_size=3, max_size=3)
)
def test_euclidean_distance_commutativity(p1, p2):
    assert euclidean_distance(p1, p2) == euclidean_distance(p2, p1)


@given(
    st.lists(st.floats(min_value=-100, max_value=100), min_size=3, max_size=3),
    st.lists(st.floats(min_value=-100, max_value=100), min_size=3, max_size=3)
)
def test_manhattan_distance_commutativity(p1, p2):
    assert manhattan_distance(p1, p2) == manhattan_distance(p2, p1)


ipytest.run(raise_on_error=True)


[0.0, 0.0, 0.0] [0.0, 0.0, 0.0]
[0.0, 0.0, 0.0] [0.0, 0.0, 0.0]
[0.0, 0.0, 0.0] [0.0, 0.0, 0.0]
[1.9, 5.960464477539063e-08, -0.99999] [-5e-324, -1.1754943508222875e-38, 2.225073858507e-311]
[0.0, 0.0, 0.0] [0.0, 0.0, 0.0]
[-13.287342254240423, 2.225073858507e-311, 28.08507649132957] [0.3333333333333333, 1.9, 2.225073858507e-311]
[0.0, 0.0, 0.0] [0.0, 0.0, 0.0]
[-1.175494351e-38, -1.9, 2.220446049250313e-16] [6.478147504169242, -1.1125369292536007e-308, 31.63465412149072]
[0.0, 0.0, 0.0] [0.0, 0.0, 0.0]
[-8.31294552481039, 2.225073858507203e-309, 1.1] [1.5, 1.1754943508222875e-38, 2.00001]
[0.0, 0.0, 0.0] [0.0, 0.0, 0.0]
[-90.03471517327071, 68.97873847476676, -2.220446049250313e-16] [58.526247634775196, -0.99999, -1.9]
[58.526247634775196, -0.99999, -1.9] [58.526247634775196, -0.99999, -1.9]
[0.3333333333333333, 2.225073858507203e-309, -19.09394486436878] [-1.401298464324817e-45, -5e-324, 2.225073858507203e-309]
[-1.401298464324817e-45, -5e-324, 2.225073858507203e-309] [-1.40129846432

<ExitCode.OK: 0>

### Инварианты классов

**4.2.** (1 балл)

В приведённом ниже коде для работы с банковским счётом есть следующие проблемы:

1. Снятие суммы без проверки баланса.
2. Снятие/добавление отрицательной суммы на счет.
3. В поле `balance` может быть записано всё что угодно.

Решите их путём добавления инвариантов класса `deal.inv`. Напишите тесты для всех методов класса `BankAccount`. Обязательно напишите тест, проверяющий, что баланс равен 0.2 после зачисления 0.3 и снятия 0.1 средств.

In [3]:
import deal
#import ipytest


def print_(pr):
    print(str(pr))
    return True

# AttributeError: 'BankAccountInvarianted' object has no attribute 'amount' ???????????????????????????????????
# Я поменял их местами в init и теперь исчезла balance

@deal.inv(
    lambda bankaccount: print_("1")
    and isinstance(bankaccount.balance, int | float) and bankaccount.balance >= 0 
    and isinstance(bankaccount.amount, int | float) and bankaccount.amount >= 0    
)
class BankAccount:
    def __init__(self, balance=0):
        self.balance = balance
        self.amount = 0
        
        
    def deposit(self, amount_):
        self.amount = amount_
        self.balance += self.amount
        return f"{self.amount} средств успешно зачислены на счёт."

    def withdraw(self, amount_):
        self.amount = amount_
        self.balance -= self.amount
        return f"{self.amount} средств успешно сняты с счёта."

    def check_balance(self):
        return f"Баланс счёта: {self.balance}"


bankaccount = BankAccount(10)
print(bankaccount.deposit(10))
print(bankaccount.check_balance())

"""
# Тесты
def test_bank_account():
    account = BankAccount()
    assert account.check_balance() == "Баланс счёта: 0"
    
    assert account.deposit(0.3) == "0.3 средств успешно зачислены на счёт."
    assert account.check_balance() == "Баланс счёта: 0.3"
    
    assert account.withdraw(0.1) == "0.1 средств успешно сняты с счёта."
    assert account.check_balance() == "Баланс счёта: 0.2"
    
    try:
        account.withdraw(0.5)
    except deal.InvContractError:
        pass
    else:
        assert False, "Ожидается ошибка при попытке снять больше, чем есть на счете"
    
    try:
        account.deposit(-0.1)
    except deal.InvContractError:
        pass
    else:
        assert False, "Ожидается ошибка при попытке зачислить отрицательную сумму"
    
    try:
        account.withdraw(-0.1)
    except deal.InvContractError:
        pass
    else:
        assert False, "Ожидается ошибка при попытке снять отрицательную сумму"
"""

# Первый способ - костыль
# https://deal.readthedocs.io/basic/values.html?highlight=deal.inv#deal-inv
"""
Invariant check condition in the next cases:
 Before class method execution.
 After class method execution.
 After some class attribute setting.
"""

#ipytest.run(raise_on_error=True)

1


AttributeError: 'BankAccountInvarianted' object has no attribute 'amount'

In [81]:
import deal
import ipytest


def print_(pr):
    print(str(pr))
    return True


@deal.inv(
    lambda bankaccount: isinstance(bankaccount.balance, int | float)
    and bankaccount.balance >= 0
)
class BankAccount:
    def __init__(self, balance=0):
        self.balance = balance
    
    @deal.pre(lambda self, amount: isinstance(amount, int | float) and amount > 0 )
    def deposit(self, amount):
        self.balance += amount
        return f"{amount} средств успешно зачислены на счёт."
    
    @deal.pre(lambda self, amount: amount > 0 and self.balance >= amount)
    def withdraw(self, amount):
        self.balance -= amount
        return f"{amount} средств успешно сняты с счёта."

    def check_balance(self):
        return f"Баланс счёта: {self.balance}"


bankaccount = BankAccount(10)
bankaccount.deposit(10)


# Тесты
def test_bank_account():
    account = BankAccount()
    assert account.check_balance() == "Баланс счёта: 0"
    
    assert account.deposit(0.3) == "0.3 средств успешно зачислены на счёт."
    assert account.check_balance() == "Баланс счёта: 0.3"
    
    assert account.withdraw(0.1) == "0.1 средств успешно сняты с счёта."
    assert account.check_balance() == f"Баланс счёта: {0.3 - 0.1}"
    


# Второй способ - добавить deal.pre и тд...
# https://deal.readthedocs.io/basic/values.html?highlight=deal.inv#deal-inv
"""
Invariant check condition in the next cases:
 Before class method execution.
 After class method execution.
 After some class attribute setting.
"""

ipytest.run(raise_on_error=True)


======================================= test session starts =======================================
platform win32 -- Python 3.10.2, pytest-7.3.1, pluggy-1.0.0
rootdir: c:\Users\iyuna\source\repos\python\kispython\solved\pract_git
plugins: hypothesis-6.75.3, typeguard-2.13.3
collected 1 item

t_6ef4bde501f349a5a447c12388f8eaf7.py .                                                      [100%]

======================================== 1 passed in 0.01s ========================================


<ExitCode.OK: 0>

In [4]:
import deal


#@deal.inv(lambda bankaccount: bankaccount.balance >= 0 and isinstance(bankaccount.balance, int | float)
class BankAccount():
    #@deal.inv(lambda self, amount: isinstance(amount, int | float))
    def __init__(self, balance=0):
        self.balance = balance

    #@deal.inv(lambda self, amount: isinstance(amount, int | float) and amount > 0 )
    def deposit(self, amount):
        self.balance += amount
        return f"{amount} средств успешно зачислены на счёт."

    #@deal.inv(lambda self, amount: amount > 0 and self.balance >= amount)
    def withdraw(self, amount):
        self.balance -= amount
        return f"{amount} средств успешно сняты с счёта."

    def check_balance(self):
        return f"Баланс счёта: {self.balance}"


In [17]:
class BankAccount:
    def __init__(self, balance=0):
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount
        return f"{amount} средств успешно зачислены на счёт."

    def withdraw(self, amount):
        self.balance -= amount
        return f"{amount} средств успешно сняты с счёта."

    def check_balance(self):
        return f"Баланс счёта: {self.balance}"

### Мутационное тестирование

**4.3.** (1 балл)

Суть мутационного тестирования (mutation testing) состоит в привнесении ошибок ("мутаций") в исходную программу (тем самым создаются "программы-мутанты"), с тем, чтобы оценить, сможет ли существующий набор тестов эти ошибки определить. Если набор тестов успешно выполняется на программе-мутанте, то это означает, что используемые тесты нуждаются в развитии.

Продолжение раздела "Библиотеки pytest и coverage (задача 2.1.)". Намеренно внесите минимум по одному ошибочному изменению в коде методов следующих классов:


1.   User - `authenticate()`, `authorize()`.
2.   Project - `add_task()`, `remove_task()`, `find_task_by_name()`, `find_task_by_start_date()`, `is_completed()`.
3.   Task - `change_status()`, `remove_performer()`.

Проверьте существующий набор тестов для задачи 2.1. на обновлённом коде. Проходят ли какие-то тесты? Попытайтесь добиться обратного эффекта: ни один тест, проверяющий указанные выше методы, не должен проходить. Проверку возможно осуществлять также через `pytest` и `coverage`.